In [ ]:
import pytesseract
import os
from PIL import Image
from googletrans import Translator
from flask import Flask, request, render_template_string

app = Flask(__name__)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

LANGUAGES = {
    "en": "English",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
    "zh-cn": "Chinese (Simplified)",
    "ar": "Arabic",
    
}


def perform_ocr(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text


def translate_text(text, target_language):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language).text
    return translated_text

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        
        if "image" not in request.files:
            return render_template_string(TEMPLATE, error="No image file provided.", languages=LANGUAGES)

        image = request.files["image"]

        
        if image.filename == "":
            return render_template_string(TEMPLATE, error="Please select a valid image file.", languages=LANGUAGES)

        if image:
            
            if not os.path.exists(app.config['UPLOAD_FOLDER']):
                os.makedirs(app.config['UPLOAD_FOLDER'])

            
            image_path = os.path.join(app.config['UPLOAD_FOLDER'], image.filename)
            image.save(image_path)

            
            ocr_result = perform_ocr(image_path)

    
            target_language = request.form.get("target_language", "en") 

            
            translated_text = translate_text(ocr_result, target_language)

            
            return render_template_string(
                TEMPLATE,
                ocr_result=ocr_result,
                translated_text=translated_text,
                languages=LANGUAGES
            )

    return render_template_string(TEMPLATE, languages=LANGUAGES)

# Embedded HTML template with enhanced styling
TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>OCR and Translate</title>
    <style>
        body {
            font-family: 'Arial', sans-serif;
            background-color: #f4f4f9;
            color: #333;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .container {
            background-color: #ffffff;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            width: 100%;
            max-width: 500px;
        }
        h1 {
            text-align: center;
            color: #4CAF50;
        }
        label {
            font-size: 1.1em;
            margin-bottom: 10px;
            display: block;
        }
        input[type="file"], select {
            width: 100%;
            padding: 10px;
            margin: 10px 0;
            border-radius: 5px;
            border: 1px solid #ccc;
        }
        button {
            width: 100%;
            padding: 12px;
            background-color: #4CAF50;
            color: white;
            font-size: 1.2em;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background-color: #45a049;
        }
        .result {
            margin-top: 20px;
            padding: 15px;
            background-color: #e9f7e9;
            border-radius: 5px;
        }
        .result h2 {
            margin-bottom: 10px;
        }
        .error {
            color: red;
            text-align: center;
            margin-top: 20px;
        }
    </style>
</head>
<body>

    <div class="container">
        <h1>Upload Image for OCR and Translation</h1>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        
        <form method="post" enctype="multipart/form-data">
            <label for="image">Select an Image:</label>
            <input type="file" name="image" id="image" accept="image/*" required><br>
            
            <label for="target_language">Select Target Language:</label>
            <select name="target_language" id="target_language">
                {% for code, language in languages.items() %}
                    <option value="{{ code }}">{{ language }}</option>
                {% endfor %}
            </select><br>
            
            <button type="submit">Submit</button>
        </form>

        {% if ocr_result %}
            <div class="result">
                <h2>OCR Result:</h2>
                <p>{{ ocr_result }}</p>
                
                <h2>Translated Text:</h2>
                <p>{{ translated_text }}</p>
            </div>
        {% endif %}
    </div>

</body>
</html>
"""

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)
